In [2]:
import praw
import pandas as pd
from tqdm.notebook import tqdm

#### DATA

In [3]:
secret='iFYDcYUNWwgFPAEwk6_vmA3ZQ2WIbQ'
client_id='uCLonTkFBuYPaFa_1cRZ3Q'
user_agent='Love Letters From Macao'
username='Existing_Dog5489'
password='kabar bahy'

In [4]:
# Read-only instance
reddit_read_only = praw.Reddit(client_id=client_id,         # your client id
                               client_secret=secret,      # your client secret
                               user_agent=user_agent)        # your user agent
 
# Authorized instance
reddit_authorized = praw.Reddit(client_id=client_id,         # your client id
                               client_secret=secret,      # your client secret
                               user_agent=user_agent,        # your user agent
                                username=username,        # your reddit username
                                password=password)     

Version 7.6.1 of praw is outdated. Version 7.7.0 was released Saturday February 25, 2023.


In [5]:
subreddit = reddit_read_only.subreddit("wallstreetbets")
 
for post in subreddit.hot(limit=5):
    print(post.title)
    print()

What Are Your Moves Tomorrow, March 14, 2023

Most Anticipated Earnings Releases for the week beginning March 13th, 2023

Live from The US Treasury

I can’t wait until March 22.

First Republic down 60% premarket



In [13]:
posts = subreddit.top("month")
# Scraping the top posts of the current month
 
posts_dict = {"Title": [], "Post Text": [],
              "ID": [], "Score": [],
              "Total Comments": [], "Post URL": []
              }
 
for post in posts:
    # Title of each post
    posts_dict["Title"].append(post.title)
     
    # Text inside a post
    posts_dict["Post Text"].append(post.selftext)
     
    # Unique ID of each post
    posts_dict["ID"].append(post.id)
     
    # The score of a post
    posts_dict["Score"].append(post.score)
     
    # Total number of comments inside the post
    posts_dict["Total Comments"].append(post.num_comments)
     
    # URL of each post
    posts_dict["Post URL"].append(post.url)
 
# Saving the data in a pandas dataframe
top_posts = pd.DataFrame(posts_dict)
top_posts


<ipython-input-13-f2aa57c2384a>:1: DeprecationWarning: Positional arguments for 'BaseListingMixin.top' will no longer be supported in PRAW 8.
Call this function with 'time_filter' as a keyword argument.
  posts = subreddit.top("month")


KeyboardInterrupt: 

In [43]:
########################################
#   CREATING DICTIONARY TO STORE THE DATA WHICH WILL BE CONVERTED TO A DATAFRAME
########################################

#   NOTE: ALL THE POST DATA AND COMMENT DATA WILL BE SAVED IN TWO DIFFERENT
#   DATASETS AND LATER CAN BE MAPPED USING IDS OF POSTS/COMMENTS AS WE WILL 
#   BE CAPTURING ALL IDS THAT COME IN OUR WAY

# SCRAPING CAN BE DONE VIA VARIOUS STRATEGIES {HOT,TOP,etc} we will go with keyword strategy i.e using search a keyword

query = ['what stock should I buy'] ## put here main stocks

for item in query:
    post_dict = {
        "title" : [],   #title of the post
       # "score" : [],   # score of the post
       # "id" : [],      # unique id of the post
       # "url" : [],     #url of the post
       # "comms_num": [],   #the number of comments on the post
       # "created" : [],  #timestamp of the post
        "body" : []         # the descriptionof post
    }
    comments_dict = {
       # "comment_id" : [],      #unique comm id
       # "comment_parent_id" : [],   # comment parent id
        "comment_body" : [],   # text in comment
       # "comment_link_id" : []  #link to the comment
    }
    for submission in tqdm(subreddit.search(query,sort = "top", time_filter="week", limit = 15)):
        post_dict["title"].append(submission.title)
       # post_dict["score"].append(submission.score)
       # post_dict["id"].append(submission.id)
       # post_dict["url"].append(submission.url)
       # post_dict["comms_num"].append(submission.num_comments)
       # post_dict["created"].append(submission.created)
        post_dict["body"].append(submission.selftext)
        
        ##### Acessing comments on the post
        submission.comments.replace_more(limit = 10)
        for comment in submission.comments.list():
#            comments_dict["comment_id"].append(comment.id)
#            comments_dict["comment_parent_id"].append(comment.parent_id)
            comments_dict["comment_body"].append(comment.body)
#            comments_dict["comment_link_id"].append(comment.link_id)
    
    post_comments = pd.DataFrame(comments_dict)

#    post_comments.to_csv('wallstreetbets'+"_comments_"+ item +"subreddit.csv")
#    post_data = pd.DataFrame(post_dict)
#    post_data.to_csv('wallstreetbets'+"_"+ item +"subreddit.csv")

0it [00:00, ?it/s]

PRELIMINARY MODEL

In [ ]:
all_stocks = pd.read_csv('list_of_stocks.csv')
all_stocks = all_stocks[all_stocks['Country'] == 'United States'].reset_index(drop=True)
all_stocks = all_stocks[['Symbol', 'Name']]

In [6]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [83]:
BLACKLIST = ['ev', 'covid', 'etf', 'nyse', 'sec', 'spac', 'fda', 'treasury', 'eod', 'fed', 'wsb', ]

def get_orgs(text):
    doc = nlp(text)
    org_list = []
    for entity in doc.ents:
        # here we modify the original code to check that entity text is not equal to one of our 'blacklisted' organizations
        # (we also add .lower() to lowercase the text, this allows us to match both 'nyse' and 'NYSE' with just 'nyse')
        if entity.label_ == 'ORG' and entity.text.lower() not in BLACKLIST:
            org_list.append(entity.text.lower())
    # if organization is identified more than once it will appear multiple times in list
    # we use set() to remove duplicates then convert back to list
    org_list = list(set(org_list))
    return org_list


In [85]:

post_comments['organizations'] = post_comments['comment_body'].apply(get_orgs)

In [96]:
 # load the English language model

# merge organizations column into one big list
orgs = post_comments['organizations'].to_list()
orgs = [org for sublist in orgs for org in sublist]

from collections import Counter
# create dictionary of organization mention frequency
org_freq = Counter(orgs)
most_common = org_freq.most_common(100)

In [100]:
for item in most_common:
    if len(all_stocks[all_stocks['Symbol'] == item[0].upper()]) == 0:
        continue
    else:
        final_ticker = item[0].upper()
        break


In [101]:
final_ticker

'IBKR'

What to sell?

In [7]:
print(spacy.__version__)

3.4.2


### Yahoo Finance

In [46]:
import yfinance as yf


In [61]:
nyse_tickers = yf.Tickers('MSFT') 
nyse_tickers

yfinance.Tickers object <MSFT>

In [67]:
yf.Ticker('goog').fast_info.day_high

93.08000183105469

In [68]:
def history(self, period="1mo", interval="1d",
            start=None, end=None, prepost=False, actions=True,
            auto_adjust=True, back_adjust=False,
            proxy=None, rounding=False, tz=None, timeout=None, **kwargs):
    """
    :Parameters:
        period : str
            Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
            Either Use period parameter or use start and end
        interval : str
            Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            Intraday data cannot extend last 60 days
        start: str
            Download start date string (YYYY-MM-DD) or _datetime.
            Default is 1900-01-01
        end: str
            Download end date string (YYYY-MM-DD) or _datetime.
            Default is now
        prepost : bool
            Include Pre and Post market data in results?
            Default is False
        auto_adjust: bool
            Adjust all OHLC automatically? Default is True
        back_adjust: bool
            Back-adjusted data to mimic true historical prices
        proxy: str
            Optional. Proxy server URL scheme. Default is None
        rounding: bool
            Round values to 2 decimal places?
            Optional. Default is False = precision suggested by Yahoo!
        tz: str
            Optional timezone locale for dates.
            (default data is returned as non-localized dates)
        timeout: None or float
            If not None stops waiting for a response after given number of
            seconds. (Can also be a fraction of a second e.g. 0.01)
            Default is None.
        **kwargs: dict
            debug: bool
                Optional. If passed as False, will suppress
                error message printing to console.
    """

In [70]:
goog = yf.Ticker('goog')
data = goog.history()
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-02-14 00:00:00-05:00,94.660004,95.175003,92.650002,94.949997,42513100,0.0,0.0
2023-02-15 00:00:00-05:00,94.739998,97.339996,94.360001,97.099998,36964500,0.0,0.0
2023-02-16 00:00:00-05:00,95.540001,97.879997,94.970001,95.779999,35642100,0.0,0.0
2023-02-17 00:00:00-05:00,95.070000,95.750000,93.449997,94.589996,31095100,0.0,0.0
2023-02-21 00:00:00-05:00,93.239998,93.415001,92.000000,92.050003,28367200,0.0,0.0
2023-02-22 00:00:00-05:00,91.933998,92.360001,90.870003,91.800003,29891100,0.0,0.0
2023-02-23 00:00:00-05:00,92.129997,92.129997,90.010002,91.070000,32423700,0.0,0.0
2023-02-24 00:00:00-05:00,89.629997,90.129997,88.860001,89.349998,31295600,0.0,0.0
2023-02-27 00:00:00-05:00,90.089996,90.449997,89.610001,90.099998,22724300,0.0,0.0


#### MODEL

,comment_id,comment_parent_id,comment_body,comment_link_id
0,jc0d8mt,t3_11pxw4j,\n**User Report**| | | |\n:--|:--|:--|:--\n**T...,t3_11pxw4j
1,jc0f9tj,t3_11pxw4j,Holy shit SPY jumped up over 14% that day,t3_11pxw4j
2,jc0kki3,t3_11pxw4j,Bullish until March 22nd because Jerome will r...,t3_11pxw4j
3,jc0de4m,t3_11pxw4j,So buy call puts?,t3_11pxw4j
4,jc13ac9,t3_11pxw4j,If they make this period into a movie I don't ...,t3_11pxw4j
...,...,...,...,...
232,jc3u3q5,t1_jc31lpa,Unless you got a portfolio loan product they h...,t3_11qcafe
233,jc31qjd,t1_jc31k5y,I'm in for 500 shares this morning. I'll see h...,t3_11qcafe
234,jc3qq9d,t1_jc3q8ei,Homes may be damaged but fewer homes now exist...,t3_11qcafe
235,jc36005,t1_jc347af,Yeah we only have enough cash in FR to keep th...,t3_11qcafe


In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import torch

In [2]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

Downloading: 100%|██████████| 499M/499M [00:21<00:00, 23.6MB/s] 


In [5]:

text = "Good night 😊"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) positive 0.8466
2) neutral 0.1458
3) negative 0.0076
